# Preprocess 

In [1]:
# Import Libraries
import os
import nltk
import csv
import string
import glob
import numpy as np
import pandas as pd

#from gensim.parsing.porter import PorterStemmer
#from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import simple_preprocess
from gensim import corpora
from gensim import models
from gensim import similarities

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## Simpsons Dataset

### Import data

In [2]:
# Import Simpsons data
raw_df = pd.read_csv('./data/raw/simpsons_dataset.csv', sep = ',')

# Glimpse at the dataset
raw_df.head(10)

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...
5,Martin Prince,I don't think there's anything left to say.
6,Edna Krabappel-Flanders,Bart?
7,Bart Simpson,Victory party under the slide!
8,NaN,NaN
9,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!


In [3]:
# Top 10 characters by spoken words
raw_df.groupby(['raw_character_text']).count().sort_values(by=['spoken_words'], ascending=False).head(10)

,spoken_words
raw_character_text,
Homer Simpson,27850
Marge Simpson,13172
Bart Simpson,12995
Lisa Simpson,10756
C. Montgomery Burns,3077
Moe Szyslak,2808
Seymour Skinner,2385
Ned Flanders,2056
Grampa Simpson,1802


In [4]:
# Filter by main characters
simpsons_df= raw_df.loc[(raw_df['raw_character_text']=='Homer Simpson') 
                        | (raw_df['raw_character_text']=='Marge Simpson') 
                        | (raw_df['raw_character_text']=='Bart Simpson') 
                        | (raw_df['raw_character_text']=='Lisa Simpson') ].dropna()


simpsons_df.head(10)

,raw_character_text,spoken_words
1,Lisa Simpson,Where's Mr. Bergstrom?
3,Lisa Simpson,That life is worth living.
7,Bart Simpson,Victory party under the slide!
9,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!
11,Lisa Simpson,Do you know where I could find him?
13,Lisa Simpson,"The train, how like him... traditional, yet en..."
15,Lisa Simpson,"I see he touched you, too."
17,Bart Simpson,"Hey, thanks for your vote, man."
19,Bart Simpson,"Well, you got that right. Thanks for your vote..."
21,Bart Simpson,"Well, don't sweat it. Just so long as a couple..."


### Process Data

In [5]:
def process_dataset(words, tags):
    """ Applies standard pre-processing to given dataset
    
    Args:
        words (str): List of strings to be processed.
        tags (str): List of tags to be encoded.
    
    Returns:
        X_train (str): List of tokenized text.
        y_train (int): List of encoded tags.
        X_test (str):List of tokenized text.
        y_test (int): List of encoded tags.
    
    """
    # Simple preprocess to words

    # Glipse at preprocess
    print('Example of a tokenized line:')
    print(simple_preprocess(words[0], deacc=False, min_len=2, max_len=15))

    # Apply preprocess to the whole dataset
    X = []
    for line in words:
        #print(line)
        X.append(simple_preprocess(line, deacc=False, min_len=2, max_len=15))
        
    # Encode tags

    # Create label encoder
    le = LabelEncoder()
    le.fit(list(set(tags)))

    # Apply encoder to the whole dataset
    y = le.transform(tags)
        
    # Split dataset into training, validation and test
    X_aux, X_test, y_aux, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_aux, y_aux, test_size=0.1765, random_state=42)
    
    # Returns preprocessed dataset
    return X_train, X_val, X_test, y_train, y_val, y_test

In [6]:
# Extract text from df
simpsons_words = simpsons_df['spoken_words'].values
# Extract tags from df
simpsons_tags = simpsons_df['raw_character_text'].values

# Apply preprocess
Xs_train, Xs_val, Xs_test, ys_train, ys_val, ys_test = process_dataset(simpsons_words, simpsons_tags)

Example of a tokenized line:
['where', 'mr', 'bergstrom']


In [7]:
# Save preprocess dataset
np.savetxt('./data/simpsons/X_train.csv', Xs_train, delimiter =", ", fmt ='% s')
np.savetxt('./data/simpsons/y_train.csv', ys_train)
np.savetxt('./data/simpsons/X_val.csv', Xs_val, delimiter =", ", fmt ='% s')
np.savetxt('./data/simpsons/y_val.csv', ys_val)
np.savetxt('./data/simpsons/X_test.csv', Xs_test, delimiter =", ", fmt ='% s')
np.savetxt('./data/simpsons/y_test.csv', ys_test)

## Friends Dataset

### Import Data

In [8]:
# Import Friends data

# Text
friends_words = []
# Tag
friends_tags = []

# Dataset dir
for dirname, _, filenames in os.walk('./data/raw/friends_dataset'):
    # Files in dir
    for filename in filenames:
        # Open file
        with open(os.path.join(dirname, filename), encoding="utf8") as fp:
            # Lines in file
            line = fp.readline()
            while line:
                data  = line.strip().split(": ")
                # Verify not a blank line
                if len(data) > 1:
                    # Append tags and words
                    if data[0] == 'Monica' or data[0] == 'Joey' or data[0] == 'Chandler' or data[0] == 'Phoebe' or data[0] == 'Ross' or data[0] == 'Rachel':
                        friends_words.append(data[1])
                        friends_tags.append(data[0])
                line = fp.readline()
                

In [9]:
# Glimpse at the dataset
friends_df = pd.DataFrame()
friends_df['character'] = friends_tags
friends_df['spoken_words'] = friends_words
friends_df.head(10)

,character,spoken_words
0,Monica,There's nothing to tell! He's just some guy I ...
1,Joey,"C'mon, you're going out with the guy! There's ..."
2,Chandler,"All right Joey, be nice. So does he have a hu..."
3,Phoebe,"Wait, does he eat chalk?"
4,Phoebe,"Just, 'cause, I don't want her to go through w..."
5,Monica,"Okay, everybody relax. This is not even a date..."
6,Chandler,Sounds like a date to me.
7,Chandler,"Alright, so I'm back in high school, I'm stand..."
8,Chandler,"Then I look down, and I realize there's a phon..."
9,Joey,Instead of...?


### Process data

In [10]:
# Apply preprocess
Xf_train, Xf_val, Xf_test, yf_train, yf_val, yf_test = process_dataset(friends_words, friends_tags)

Example of a tokenized line:
['there', 'nothing', 'to', 'tell', 'he', 'just', 'some', 'guy', 'work', 'with']


In [11]:
# Save preprocess dataset
np.savetxt('./data/friends/X_train.csv', Xf_train, delimiter =", ", fmt ='% s')
np.savetxt('./data/friends/y_train.csv', yf_train)
np.savetxt('./data/friends/X_val.csv', Xf_val, delimiter =", ", fmt ='% s')
np.savetxt('./data/friends/y_val.csv', yf_val)
np.savetxt('./data/friends/X_test.csv', Xf_test, delimiter =", ", fmt ='% s')
np.savetxt('./data/friends/y_test.csv', yf_test)